# Notes on Chapter 4
## Smoothing motivation

If a word appears in the test set in a context different than any it appears in the training set, then the n-gram model will assign it a zero probability even if it is a likely word. To fix this, we want our model to give words in novel contexts a non-zero probability either by pretending that it occured in the training set (increase its count by 1, a la Laplace smoothing) or by considering it in a less-specific context (via interpolation or the various backoff strategies).

But we want our n-gram model to provide a probability mass function -- all of the probabilities should sum to 1 -- so if we increase the probability of novel n-grams then we need to decrease the probability of frequent n-grams. This is what the text means by "discounting": "To keep a language model from assigning zero probability to these unseen events, we’ll have to shave off a bit of probability mass from some more frequent events and give it to the events we’ve never seen."

So we "smooth" the probability mass function: novel n-grams are made more probable and frequent n-grams less probable, with the sum of all probabilities = 1.

However, it's possible we don't care if it is a true probability distribution. That's the the point of the "Stupid Backup" strategy: it's probabilities don't sum to 1, but it is simple because it doesn't have to worry about discounting.

## Notation

Many of the formulas are given without much in the way of explanation -- did this cause some of the confusion in class?

I was confused at first by this bit of equation 4-26:

$$\lambda_n \left(w_{n-2}^{n-1}\right)$$

I thought it was multiplying lambda by a sequence of words and didn't know what that meant. But the parentheses in this case signify a paramter: so it means the lambda value for the given bigram.

# Excerices 4.8-4.11

## 4.8

> Write a program to compute unsmoothed unigrams and bigrams.

Two possible implementations come to mind.

One is to store n-grams in a [trie structure](https://en.wikipedia.org/wiki/Trie) where each node represents a word and contains both a count and a `children` dict whose keys are strings and whose values are nodes representing words which have followed the current node in an n-gram. So this sentence:

    "<s>a b a c</s>"

would become a forest of bigram trees (where the count is in parentheses):

    <s>(1)     a(2)     b(1)  c(1)
      |       /   \      |     |
     a(1)   b(1)   c(1) a(1) </s>(1)

The top-level nodes are the unigrams, the second level nodes following their parent are the bigrams, and so on.

The other implementation I thought of is to build a dictionary of whose keys are n-tuples of strings representing the n-gram and whose values are the count of times that n-gram appeared in the training text. We could then also build an n-1-gram dict, an n-2-gram dict... and a unigram dict, and store them all in a dict keyed by `n`:

````python
ngrams_dict = {
    1: { (word1,): 5,          # unigrams
         (word2,): 4
       },
    2: { (word1, word2): 3,    # bigrams
         (word2, word3): 2
       },
    ...,
    n: ...                     # n-grams
}
````

The trie would take less memory which is an important consideration for very large text corpuses. But the dict-of-ngrams method seems easier to implement so I decided to go with that one.

In [1]:
from collections import Counter
from random import random
from math import log2
SENT_START = "<s>"
SENT_END = "</s>"

In [2]:
class NGramModel:
    """
    Class representing an n-gram model of a language.
    
    Initialize it with some sentences and a degree (default=2, for bigrams),
    and it will count all n-grams, n-1-grams ... unigrams.
    """
    def __init__(self, sentences, n=2):
        """
        Parameters:
            sentences: a list of sentences (each sentence a list of words);
                       SENT_START and SENT_END sentinals are added before counting n-grams
                       
            n: an integer specifying the highest-order n-grams to count
        
        More sentences can be added to the model after initialization by callig add_sentences()
        """
        self.n = n
        self.ngram_dict = {}
        self.add_sentences(sentences) 
    
    def add_sentences(self, sentences):
        """
        Parameters:
            sentences: a list of sentences (each sentence a list of words);
                       SENT_START and SENT_END sentinals are added before counting n-grams
        """
        # build a list of the nth-order ngrams
        ngrams = []
        for sentence in sentences:
            # Add sentinal symbols to start and end of each sentence
            # we add n-number of symbols so that the sentence ['a', 'b', 'c'] becomes
            # ['<s>', '<s>', 'a', 'b', 'c', '</s>', '</s>'] for n=2
            # 
            # See "Some practical issues" on pages 6-7 of the text.
            n = self.n
            start_tokens = [SENT_START]*n
            end_tokens = [SENT_END]*n
            sentence[0:0] = start_tokens
            sentence.extend(end_tokens)
            length = len(sentence)
            for i in range(length):
                # from through the words in sentence left-to-right taking n at a time
                span = i+n
                if span > length:
                    # got to the end, move to next sentence
                    break
                ngrams.append(tuple(sentence[i:span]))
        
        # build ngrams dict
        # This does one thing clever: it derives all the lower-order n-grams from
        # the n-gram tuples instead of processing the sentences a second time.
        #
        # To create a bigram from ('word1', 'word2', 'word3') for example, just remove the last component.
        for i in reversed(range(n)):
            degree = i+1
            lower = n - degree
            degn_grams = [ngram[0:n-lower] for ngram in ngrams]
            counts = Counter(degn_grams)
            
            # Now merge these counts with the existing counts in self.ngram_dict[degree]
            for k, v in counts.items():
                try:
                    # make sure the degree key exists
                    self.ngram_dict[degree]
                except KeyError:
                    self.ngram_dict[degree] = {}
                try:
                    # if the key for this ngram doesn't exist, then create it
                    self.ngram_dict[degree][k] += v
                except KeyError:
                    self.ngram_dict[degree][k] = v
        return ngrams
    
    @classmethod
    def ngram_to_tuple(cls, ngram):
        """
        Helper method to turn strings and lists into tuples.
        """
        if isinstance(ngram, str):
            # if we were passed a bare string, make it a list
            ngram = ngram.split()
        if isinstance(ngram, list):
            # if we were passed a list, make it a tuple
            ngram = tuple(ngram)
        return ngram
    
    def ngrams(self, n=None):
        """
        Return a list of n-grams of order n
        """
        if n is None:
            # default to the n we were initialized with
            n = self.n
        return self.ngram_dict[n].keys()
    
    def count(self, ngram):
        """
        Returns the count for ngram or None if ngram does not exist in model.
        """
        ngram = self.ngram_to_tuple(ngram)
        order = len(ngram)
        if order > self.n:
            raise ValueError("The order of the given ngram exceeds the order of this model.")
        return self.ngram_dict[order].get(ngram, None)
    
    def prob(self, ngram):
        """
        Return the MLE of the probability for ngram.
        Unsmoothed (if ngram doesn't exist in the model, its probability is 0)
        """
        ngram = self.ngram_to_tuple(ngram)
        count = self.count(ngram)
        if count is None: return 0
        
        # get the count of the n-1th word in the n-gram
        n_minus_one = len(ngram)-1
        if n_minus_one == 0:
            # for unigrams, we get the count of all word types:
            pcount = sum(self.ngram_dict[1].values())
        else:
            pcount = self.count(ngram[0:n_minus_one])
        
        return count/pcount
    
    def top(self, x=10, n=None, prob=None):
        """
        Return a list of tuples containing the x most frequent n-grams
        """
        if n is None: n = self.n
        if prob is None: prob = self.prob

        counts = self.ngram_dict[n]
        return sorted(counts, key=counts.get, reverse=True)[0:x]

    def format_top(self, x=10, n=None, prob=None):
        """
        Returns a string of the x most frequent n-grams, their counts (c) and their probabilities (p)
        """
        if prob is None: prob = self.prob
        top_list = self.top(x=x, n=n, prob=prob)
        
        string = ''
        for ngram in top_list:
            c = self.count(ngram)
            p = prob(ngram)
            string += "%s: c=%d, p=%f\n" % (ngram, c, p)
        return string

To show how the `NGramModel` class works:

In [3]:
example_sentences = [
    [*"I am Sam".split()],
    [*"Sam I am".split()],
    [*"I do not like green eggs and ham".split()]
]
print(example_sentences)

[['I', 'am', 'Sam'], ['Sam', 'I', 'am'], ['I', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham']]


In [4]:
bigram_model = NGramModel(example_sentences, 2)
bigrams = bigram_model.ngrams(2) # get a list of all bigrams
print(bigrams)

dict_keys([('Sam', '</s>'), ('am', 'Sam'), ('<s>', '<s>'), ('do', 'not'), ('am', '</s>'), ('<s>', 'Sam'), ('not', 'like'), ('</s>', '</s>'), ('Sam', 'I'), ('<s>', 'I'), ('I', 'am'), ('and', 'ham'), ('eggs', 'and'), ('like', 'green'), ('ham', '</s>'), ('I', 'do'), ('green', 'eggs')])


In [5]:
# Count how many times the bigram 'I am' appeared in training text
bigram_model.count(['I', 'am'])

2

In [6]:
# find the MLE probability that the bigram the word 'I' will follow the word 'Sam' in the language
bigram_model.prob(['Sam', 'I'])

0.5

## 4.9

> Run your N-gram program on two different small corpora of your choice (you
might use email text or newsgroups). Now compare the statistics of the two
corpora. What are the differences in the most common unigrams between the
two? How about interesting differences in bigrams?

We'll use some of the example corpuses which come with NLTK. They are already tokenized and segmented into sentences. See http://www.nltk.org/book/ch02.html

In [8]:
import nltk
hamlet = nltk.corpus.gutenberg.sents('shakespeare-hamlet.txt')
chesterton = nltk.corpus.gutenberg.sents('chesterton-thursday.txt')
bigram_hamlet = NGramModel(hamlet, 2)
bigram_chester = NGramModel(chesterton, 2)

Below we print the top 20 unigrams and bigrams with counts (c) and probabilities (p) from each corpus. The punctuation and sentence markers sort of get in the way. The 20th most frequent unigram in *The Man Who Was Thursday* is a character's name.

In [9]:
print("Top 20 unigrams, their counts, and probabilities from Hamlet:")
print(bigram_hamlet.format_top(x=20, n=1))
print()

print("Top 20 unigrams, their counts, and probabilities from The Man Who Was Thursday:")
print(bigram_chester.format_top(x=20, n=1))
print()

print("Top 20 bigrams, their counts, and probabilities from Hamlet:")
print(bigram_hamlet.format_top(x=20, n=2))
print()

print("Top 20 bigrams, their counts, and probabilities from The Man Who Was Thursday:")
print(bigram_chester.format_top(x=20, n=2))
print()

Top 20 unigrams, their counts, and probabilities from Hamlet:
('<s>',): c=6212, p=0.133082
('</s>',): c=3106, p=0.066541
(',',): c=2892, p=0.061956
('.',): c=1886, p=0.040404
('the',): c=860, p=0.018424
("'",): c=729, p=0.015618
('and',): c=606, p=0.012983
('to',): c=576, p=0.012340
('of',): c=576, p=0.012340
(':',): c=565, p=0.012104
('I',): c=553, p=0.011847
('you',): c=479, p=0.010262
('?',): c=459, p=0.009833
('a',): c=449, p=0.009619
('my',): c=435, p=0.009319
('in',): c=359, p=0.007691
('it',): c=354, p=0.007584
('Ham',): c=337, p=0.007220
('is',): c=304, p=0.006513
(';',): c=298, p=0.006384


Top 20 unigrams, their counts, and probabilities from The Man Who Was Thursday:
('<s>',): c=7484, p=0.093039
('</s>',): c=3742, p=0.046520
(',',): c=3488, p=0.043362
('the',): c=3291, p=0.040913
('.',): c=2717, p=0.033777
('a',): c=1713, p=0.021296
('of',): c=1710, p=0.021258
('and',): c=1568, p=0.019493
('"',): c=1336, p=0.016609
('to',): c=1045, p=0.012991
('in',): c=888, p=0.011039
('I',

## 4.10

> Add an option to your program to generate random sentences.

This is the fun part! We'll actually add three new methods to the NGramModel class:

- **continue_all** when given a context (a list of words whose length is less than n) will return all the possible next words

- **continue_prob** when given a context (a list of words whose length is less than n) will return a random word based on the probability model of the language

- **random_sentence** repeatedly calls continue_prob() to generate a random sentence.

In [10]:
# Extend our NGramModel with some more functions
class NGramModel(NGramModel):
    
    def continue_all(self, ngram):
        """
        Return a list of possible words following the words in the tuple ngram.
        
        For example, if our model is a trigram model, we can give this function a bigram
        (or unigram) and it will return all possible words which could follow. This takes
        words which never occured in the context of ngram as having a 0 probability of occurring
        (unsmoothed)
        """
        ngram = self.ngram_to_tuple(ngram)
        order = len(ngram)+1
        if order > self.n:
            raise ValueError("The order of the given ngram exceeds the order of this model.")
        
        results = []
        for candidate in self.ngram_dict[order].keys():
            short = candidate[0:-1]
            if short == ngram:
                results.append(candidate[-1])
        return results
    
    def continue_prob(self, ngram):
        """
        Give a random continuation to ngram
        
        Given an ngram of order less than n as context, randomly produce the next word according
        to the probability model.
        """
        ngram = self.ngram_to_tuple(ngram)
        order = len(ngram)+1
        if order > self.n:
            raise ValueError("The order of the given ngram exceeds the order of this model.")
        
        # generate a uniform random number [0, 1)
        rand = random()
        
        # variable to hold the accumulated probability
        acc = 0
        
        # Get a list of possible continuations... in a smoothed model
        # we would have to instead iterate over all ngrams
        possible = self.continue_all(ngram)
        
        # We iterate over all the possible words, accumulating their probabilities
        # until the accumulated probability is greater than the random number
        # This will produce a random word distributed according to our model's probability mass
        for word in possible:
            acc += self.prob(ngram + (word,))
            if rand < acc:
                return word

    def random_sentence(self):
        """
        Repeatedly calls continue_prob() to generate a random sentence.
        """
        words = [SENT_START] * (self.n-1)
        next_word = None
        while next_word != SENT_END:
            next_word = self.continue_prob(words[-self.n+1:])
            words.append(next_word)
            
        # remove start and end of sentence tokens:
        words = [word for word in words[:-1] if word != SENT_START]
        return ' '.join(words)

Let's try it by generating 5 random sentences from trigram models based on the two corpuses we used earlier:

In [11]:
trigram_hamlet = NGramModel(hamlet, 3)
trigram_chester = NGramModel(chesterton, 3)

In [12]:
print("Random sentences based on Hamlet:\n")
for i in range(5):
    print("%d: " % i + trigram_hamlet.random_sentence())

print()

print("Random sentences based on The Man Who Was Thursday:\n")
for i in range(5):
    print("%d: " % i + trigram_chester.random_sentence())

Random sentences based on Hamlet:

0: Affection , puh .
1: Ham .
2: To sound what stop she please .
3: Thy selfe do grace to them : There with fantasticke Garlands did she come , it courses through The naturall Gates and Allies of the knee , Where in necessitie of matter Beggard , Will want true colour ; teares perchance for blood
4: Exeunt

Random sentences based on The Man Who Was Thursday:

0: The outer ring -- the main mass of men under the influence of Syme again grew black with supernatural terrors .
1: " We might have been one to the blue and buttons !
2: " Comrades ," he said , " what are you not pull my nose ?"
3: " Oh , bring me some day ."
4: He read the message --


## 4.11

> Add an option to your program to compute the perplexity of a test set.

We add a method `perplexity()` based on this formula:

$$exp\left({\frac{1}{N}\sum_N \log \left(\frac{1}{p(w_i\vert w_{i-1})}\right)}\right)$$

Except we use log base 2 instead of natural log. Note that this is the same as equation 4.16 in the text, except it uses a sum of logprobs instead of multiplying probabilities (like the text itself recommends doing elsewhere).

In [13]:
class NGramModel(NGramModel):
    def perplexity(self, ngram_iter):
        """
        Takes an iterable of n-tuples representing n-grams and calculates the model perplexity.
        """
        sum_of_probs = 0
        N = 0
        for ngram in ngram_iter:
            N += len(ngram)
            sum_of_probs += log2(1/self.prob(ngram))
        avg = sum_of_probs / N
        return 2**avg

Becuase we haven't handled out of vocabulary words in our corpus, and we haven't implemented smoothing so that even known words in new contexts have a probability of 0, we can't really apply our perplexity measure to a real corpus (any unknown n-gram would cause a divide by zero).

But we can give it a few known n-grams to see that it works:

In [14]:
bigram_chester = NGramModel(chesterton, 2)
bigram_chester.perplexity([['<s>', 'is'], ['there', 'his']])

32.44134664616274

I'm actually not sure if it is working correctly. And the text says about calculating perplexity:

> Since this sequence will cross many sentence boundaries, we need to include the begin- and end-sentence markers `<s>` and `</s>` in the probability computation. We also need to include the end-of-sentence marker `</s>` (but not the beginning-of-sentence marker `<s>`) in the total count of word tokens N.

I guess it says not to count `<s>` because the probability that the next word in the sentence could be the first word in the sentence is 0... but I am currently counting `<s>` in both `perplexity` and in the model probabilities. I'm not sure what the ramifications are.